In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("data"))
import zipfile
import sys
import time

from keras import backend, models, layers, initializers, regularizers, constraints, optimizers
from keras import callbacks as kc
from keras import optimizers as ko

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import log_loss
import time


dense_layer_sizes = [37]
dropout_rate = 0.6
learning_rate = 0.001
n_fold = 5
batch_size = 32
epochs = 1000
patience = 100
# n_test = 100
lambd = 0.1 # L2 regularization

['sample_submission_stage_1.csv', 'test_stage_1.tsv', 'trees']


Using TensorFlow backend.


In [2]:
def build_mlp_model(input_shape):
	X_input = layers.Input(input_shape)
	X = layers.Dropout(dropout_rate, seed = 7)(X_input)
	# First dense layer
	X = layers.Dense(dense_layer_sizes[0], name = 'dense0')(X)
	X = layers.BatchNormalization(name = 'bn0')(X)
	X = layers.Activation('relu')(X)
	X = layers.Dropout(dropout_rate, seed = 7)(X)

	# Output layer
	X = layers.Dense(3, name = 'output', kernel_regularizer = regularizers.l2(lambd))(X)
	X = layers.Activation('softmax')(X)

	# Create model
	model = models.Model(input = X_input, output = X, name = "classif_model")
	return model

In [3]:
def parse_json(embeddings, embedding_size):
    '''
    Parses the embeddigns given by BERT, and suitably formats them to be passed to the MLP model

    Input: embeddings, a DataFrame containing contextual embeddings from BERT, as well as the labels for the classification problem
    columns: "emb_A": contextual embedding for the word A
             "emb_B": contextual embedding for the word B
             "emb_P": contextual embedding for the pronoun
             "label": the answer to the coreference problem: "A", "B" or "NEITHER"

    Output: X, a numpy array containing, for each line in the GAP file, the concatenation of the embeddings of the target words
            Y, a numpy array containing, for each line in the GAP file, the one-hot encoded answer to the coreference problem
    '''
    embeddings.sort_index(inplace = True) # Sorting the DataFrame, because reading from the json file messed with the order
    X = np.zeros((len(embeddings),3* embedding_size))
    Y = np.zeros((len(embeddings), 3))

    # Concatenate features
    for i in range(len(embeddings)):
        A = np.array(embeddings.loc[i,"emb_A"])
        B = np.array(embeddings.loc[i,"emb_B"])
        P = np.array(embeddings.loc[i,"emb_P"])
        X[i] = np.concatenate((A,B,P))

    # One-hot encoding for labels
    for i in range(len(embeddings)):
        label = embeddings.loc[i,"label"]
        if label == "A":
            Y[i,0] = 1
        elif label == "B":
            Y[i,1] = 1
        else:
            Y[i,2] = 1

    return X, Y

In [4]:
dev_filename = "contextual_embeddings_gap_development.json"
val_filename = "contextual_embeddings_gap_validation.json"
test_filename = "contextual_embeddings_gap_test.json"

In [5]:
# Keep all BERTs learn on the same corpus
remove_test = []
remove_validation = []
remove_development = [209, 1506, 1988]

In [9]:
def train_folds(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold, model_tag,
                dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
                oof_folder='oof/', pred_folder='outputs/'):
    '''
    Arguments:
        tag (data_tag): the tag of model and vec
        embedding_size: the size of bert embedding
        layer: to indicate which layer of bert is going to use
        checkpoint_path: the path to model_checkpoint folder
        n_fold: the number of CV folds
        model_tag: the prefix of the prediction and oof file and model chekpoint
    '''

    tag = tag + str(layer) # follow the original naming style
    dev_filename = tag + dev_filename
    val_filename = tag + val_filename
    test_filename = tag + test_filename
    
    development = pd.read_json(os.path.join(dev_folder_path, dev_filename))
    X_development, Y_development = parse_json(development, embedding_size)

    validation = pd.read_json(os.path.join(val_folder_path, val_filename))
    X_validation, Y_validation = parse_json(validation, embedding_size)

    test = pd.read_json(os.path.join(test_folder_path, test_filename))
    X_test, Y_test = parse_json(test, embedding_size)

    # There may be a few NaN values, where the offset of a target word is greater than the max_seq_length of BERT.
    # They are very few, so I'm just dropping the rows.
    # remove_test = [row for row in range(len(X_test)) if np.sum(np.isnan(X_test[row].reshape(-1)))]
    X_test = np.delete(X_test, remove_test, 0)
    Y_test = np.delete(Y_test, remove_test, 0)

    # remove_validation = [row for row in range(len(X_validation)) if np.sum(np.isnan(X_validation[row].reshape(-1)))]
    X_validation = np.delete(X_validation, remove_validation, 0)
    Y_validation = np.delete(Y_validation, remove_validation, 0)

    # We want predictions for all development rows. So instead of removing rows, make them 0
    # remove_development = [row for row in range(len(X_development)) if np.sum(np.isnan(X_development[row].reshape(-1)))]
    X_development = np.delete(X_development, remove_development, 0)
    Y_development = np.delete(Y_development, remove_development, 0)
    
    # Will train on data from the gap-test and gap-validation files, in total 2454 rows
    X_train = np.concatenate((X_test, X_validation, X_development), axis = 0)
    Y_train = np.concatenate((Y_test, Y_validation, Y_development), axis = 0)

    # Will predict probabilities for data from the gap-development file; initializing the predictions
    prediction = np.zeros((len(X_development), 3)) # testing predictions

    # Training and cross-validation
    folds = KFold(n_splits=n_fold, shuffle=True, random_state=3)
    scores = []
    oof = np.zeros_like(Y_train)
    
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train)):
        # split training and validation data
        print('Fold', fold_n, 'started at', time.ctime())
        X_tr, X_val = X_train[train_index], X_train[valid_index]
        Y_tr, Y_val = Y_train[train_index], Y_train[valid_index]

        # Define the model, re-initializing for each fold
        classif_model = build_mlp_model([X_train.shape[-1]])
        classif_model.compile(optimizer=optimizers.Adam(lr=learning_rate), loss="categorical_crossentropy")

        callbacks = [
                     kc.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
                     kc.ModelCheckpoint(os.path.join(checkpoint_path, model_tag + tag + str(fold_n) + '.pt'), monitor='val_loss', verbose=0, save_best_only=True, mode='min')]

        # train the model
        classif_model.fit(x=X_tr, y=Y_tr, epochs=epochs, batch_size=batch_size, 
                          callbacks=callbacks, validation_data=(X_val, Y_val), verbose=0)

        # make predictions on validation and test data
        pred_valid = classif_model.predict(x=X_val, verbose=0)
        oof[valid_index] = pred_valid
        pred = classif_model.predict(x=X_development, verbose=0)

        # oof[valid_index] = pred_valid.reshape(-1,)
        scores.append(log_loss(Y_val, pred_valid))
        prediction += pred
    
    prediction /= n_fold    
    
    print("-" * 30)
    print("For the model", tag)
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    print(scores)
    
    # Write the prediction to file for submission
    oof_df = pd.DataFrame(oof)
    oof_df.to_csv(oof_folder + model_tag + tag + ".csv", index=False)

# Bert Base
## Uncased

In [7]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_base"
embedding_size = 768
layer = 8
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-base-uncased-seq512-"
pred_tag = "naive-"

In [8]:
train_folds(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='outputs/', model_tag=pred_tag)

Fold 0 started at Mon Apr 15 14:30:58 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`
  from ipykernel import kernelapp as app


Fold 1 started at Mon Apr 15 14:32:27 2019
Fold 2 started at Mon Apr 15 14:33:55 2019
Fold 3 started at Mon Apr 15 14:35:15 2019
Fold 4 started at Mon Apr 15 14:36:32 2019
Fold 5 started at Mon Apr 15 14:38:04 2019
Fold 6 started at Mon Apr 15 14:40:07 2019
------------------------------
For the model bert-base-uncased-seq512-8
CV mean score: 0.4393, std: 0.0198.
[0.4528021516174784, 0.4419618785703496, 0.422456534245503, 0.4218643427577846, 0.4103121842551371, 0.4633743767757231, 0.4626042989704471]
Test score: 0.22137542657638595
------------------------------


ValueError: Length of values does not match length of index

## Cased

In [10]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_base_cased"
embedding_size = 768
layer = 8
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-base-cased-seq512-"
pred_tag = "naive-"

In [11]:
train_folds(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='outputs/', model_tag=pred_tag)

Fold 0 started at Mon Apr 15 14:43:29 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`
  from ipykernel import kernelapp as app


Fold 1 started at Mon Apr 15 14:44:23 2019
Fold 2 started at Mon Apr 15 14:45:26 2019
Fold 3 started at Mon Apr 15 14:46:30 2019
Fold 4 started at Mon Apr 15 14:47:30 2019
Fold 5 started at Mon Apr 15 14:48:46 2019
Fold 6 started at Mon Apr 15 14:49:36 2019
------------------------------
For the model bert-base-cased-seq512-8
CV mean score: 0.4905, std: 0.0260.
[0.4972360841906117, 0.46110386217475, 0.4736347498743591, 0.49113536845971745, 0.4619680452013188, 0.5084957606328465, 0.5395958285293162]
Test score: 0.2906273791398692
------------------------------


# Bert Big
## Uncased

In [12]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_big"
embedding_size = 1024
layer = 19
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-large-uncased-seq300-"
pred_tag = "naive-"

In [13]:
train_folds(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='outputs/', model_tag=pred_tag)

Fold 0 started at Mon Apr 15 14:51:11 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`
  from ipykernel import kernelapp as app


Fold 1 started at Mon Apr 15 14:51:58 2019
Fold 2 started at Mon Apr 15 14:53:08 2019
Fold 3 started at Mon Apr 15 14:54:27 2019
Fold 4 started at Mon Apr 15 14:55:58 2019
Fold 5 started at Mon Apr 15 14:56:57 2019
Fold 6 started at Mon Apr 15 14:57:45 2019
------------------------------
For the model bert-large-uncased-seq300-19
CV mean score: 0.3805, std: 0.0313.
[0.37305250019210306, 0.3738718491489601, 0.37252183842618986, 0.3491585163411635, 0.34228276159344245, 0.42119303584404866, 0.4313765856804964]
Test score: 0.17803088601250697
------------------------------


## Cased

In [14]:
dev_folder_path = val_folder_path = test_folder_path = "vector/bert_big_cased"
embedding_size = 1024
layer = 18
checkpoint_path = "stage_1_checkpoints/"
n_fold = 7
tag = "bert-large-cased-seq300-"
pred_tag = "naive-"

In [15]:
train_folds(tag, dev_folder_path, val_folder_path, test_folder_path, embedding_size, layer, checkpoint_path, n_fold,
            dev_filename=dev_filename, val_filename=val_filename, test_filename=test_filename,
            oof_folder='oof/', pred_folder='outputs/', model_tag=pred_tag)

Fold 0 started at Mon Apr 15 14:58:37 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`
  from ipykernel import kernelapp as app


Fold 1 started at Mon Apr 15 14:59:26 2019
Fold 2 started at Mon Apr 15 15:00:32 2019
Fold 3 started at Mon Apr 15 15:01:37 2019
Fold 4 started at Mon Apr 15 15:02:27 2019
Fold 5 started at Mon Apr 15 15:03:44 2019
Fold 6 started at Mon Apr 15 15:04:40 2019
------------------------------
For the model bert-large-cased-seq300-18
CV mean score: 0.4190, std: 0.0237.
[0.4233008980611559, 0.4071883598085637, 0.40699723165057805, 0.41427951452648853, 0.37972288588196007, 0.4472983241099176, 0.454350395469127]
Test score: 0.22459776728373343
------------------------------
